In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

In [2]:
'''
데이터 파일 로드
'''
Gyeonggi = pd.read_csv('경기.CSV',encoding='utf-8')
Seoul = pd.read_csv('서울.csv',encoding='utf-8')
incheon = pd.read_csv('인천.csv',encoding='utf-8')

In [3]:
store_list=['gs','cu','세븐','미니','마트','씨유','수퍼','storyway','마트','mart']

In [14]:
incheon=incheon[['상호명','상권업종대분류명','상권업종중분류명','상권업종소분류명','표준산업분류코드','표준산업분류명','도로명','경도', '위도']]

In [15]:
incheon['상권업종소분류명'][1].upper()

'여성미용실'

In [16]:
incheon_con=incheon[incheon['표준산업분류명'] == '체인화 편의점']

In [17]:
incheon_con

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류코드,표준산업분류명,도로명,경도,위도
44,GS25,소매,종합소매점,편의점,G47122,체인화 편의점,인천광역시 부평구 시장로20번길,126.725887,37.492558
66,CU,소매,종합소매점,편의점,G47122,체인화 편의점,인천광역시 미추홀구 경원대로,126.689896,37.461313
110,훼미리마트가좌현우,소매,종합소매점,편의점,G47122,체인화 편의점,인천광역시 서구 가정로,126.671807,37.496008
144,CU,소매,종합소매점,편의점,G47122,체인화 편의점,인천광역시 서구 장고개로337번길,126.684388,37.493121
153,CU,소매,종합소매점,편의점,G47122,체인화 편의점,인천광역시 미추홀구 경인로268번길,126.670030,37.461659
...,...,...,...,...,...,...,...,...,...
120557,코리아세븐주안우리들점,소매,종합소매점,편의점,G47122,체인화 편의점,인천광역시 미추홀구 주안로,126.683910,37.463625
120646,GS25,소매,종합소매점,편의점,G47122,체인화 편의점,인천광역시 남동구 구월말로,126.724839,37.455064
120797,CU,소매,종합소매점,편의점,G47122,체인화 편의점,인천광역시 미추홀구 경원대로780번길,126.692626,37.453865
120830,이마트,소매,종합소매점,편의점,G47122,체인화 편의점,인천광역시 서구 검단로501번길,126.658846,37.603461


In [ ]:
incheon_con

In [4]:
'''
서울, 경기, 인천 편의점
1. 편의점만 남겨두고 제거
2. 필요한 열을 제외하고 삭제(경도/위도)
3. 열의 순서를 다시부여
'''

Gyeonggi_food = Gyeonggi.copy()
Seoul_food = Seoul.copy()
incheon_food = incheon.copy()
################################################################################################################
# 경기도 
Gyeonggi_food.drop(Gyeonggi_food[Gyeonggi_food['상권업종대분류명'] != "음식"].index, inplace=True)
Gyeonggi_food.drop(Gyeonggi_food[Gyeonggi_food['상권업종중분류명'] == '제과제빵떡케익'].index, inplace=True)
Gyeonggi_food.drop(Gyeonggi_food[Gyeonggi_food['상권업종중분류명'] == '음식배달서비스'].index, inplace=True)
Gyeonggi_food.drop(Gyeonggi_food[Gyeonggi_food['상권업종중분류명'] == '유흥주점'].index, inplace=True)
Gyeonggi_food.drop(Gyeonggi_food[Gyeonggi_food['상권업종중분류명'] == '커피점/카페'].index, inplace=True)

Gyeonggi_food = Gyeonggi_food[['상호명', '상권업종중분류명',
                                '상권업종소분류명','도로명','경도', '위도']]

Gyeonggi_food.reset_index(inplace=True)
Gyeonggi_food.drop(['index'],axis=1,inplace=True)
################################################################################################################
#서울
Seoul_food.drop(Seoul_food[Seoul_food['상권업종대분류명'] != "음식"].index, inplace=True)
Seoul_food.drop(Seoul_food[Seoul_food['상권업종중분류명'] == '제과제빵떡케익'].index, inplace=True)
Seoul_food.drop(Seoul_food[Seoul_food['상권업종중분류명'] == '음식배달서비스'].index, inplace=True)
Seoul_food.drop(Seoul_food[Seoul_food['상권업종중분류명'] == '유흥주점'].index, inplace=True)
Seoul_food.drop(Seoul_food[Seoul_food['상권업종중분류명'] == '커피점/카페'].index, inplace=True)

Seoul_food = Seoul_food[['상호명', '상권업종중분류명',
                                '상권업종소분류명','도로명','경도', '위도']]

Seoul_food.reset_index(inplace=True)
Seoul_food.drop(['index'],axis=1,inplace=True)
################################################################################################################
#인천
incheon_food.drop(incheon_food[incheon_food['상권업종대분류명'] != "음식"].index, inplace=True)
incheon_food.drop(incheon_food[incheon_food['상권업종중분류명'] == '제과제빵떡케익'].index, inplace=True)
incheon_food.drop(incheon_food[incheon_food['상권업종중분류명'] == '음식배달서비스'].index, inplace=True)
incheon_food.drop(incheon_food[incheon_food['상권업종중분류명'] == '유흥주점'].index, inplace=True)
incheon_food.drop(incheon_food[incheon_food['상권업종중분류명'] == '커피점/카페'].index, inplace=True)

incheon_food = incheon_food[['상호명', '상권업종중분류명',
                                '상권업종소분류명','도로명','경도', '위도']]

incheon_food.reset_index(inplace=True)
incheon_food.drop(['index'],axis=1,inplace=True)
################################################################################################################